## Dog Breed Prediction

In this project, we will see how to use Keras and TensorFlow to build, train, and test a Convolutional Neural Network capable of identifying the breed of a dog in a supplied image. This is a supervised learning problem, specifically a multiclass classification problem.

In [4]:
print("Hello")

Hello


In [5]:
pip install google.colab


  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): still running...
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [843 lines of output]
  C:\Users\Nikhil Raj\AppData\Local\Temp\pip-install-skk_rd6r\pandas_76eba4965ec946e2906f117c064604c4\setup.py:50: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    _CYTHON_INSTALLED = ver >= LooseVersion(min_cython_ver)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-3.8\pandas
  copying pandas\testing.py -> build\lib.win-amd64-3.8\pandas
  copying pandas\_version.py -> build\lib.win-amd64-3.8\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-3.8\pandas
  creating build\lib.win-amd64-3.8\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-3.8\pandas\api
  creating build\lib.win-amd64-3.8\pandas\arrays
  copying pandas\arrays\_

  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Running setup.py clean for pandas
Failed to build pandas
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Uninstalling urllib3-1.26.4:
      Successfully uninstalled urllib3-1.26.4
  Attempting uninstall: six
    Found existing ins


  copying pandas\plotting\_converter.py -> build\lib.win-amd64-3.8\pandas\plotting
  copying pandas\plotting\_core.py -> build\lib.win-amd64-3.8\pandas\plotting
  copying pandas\plotting\_misc.py -> build\lib.win-amd64-3.8\pandas\plotting
  copying pandas\plotting\_style.py -> build\lib.win-amd64-3.8\pandas\plotting
  copying pandas\plotting\_timeseries.py -> build\lib.win-amd64-3.8\pandas\plotting
  copying pandas\plotting\_tools.py -> build\lib.win-amd64-3.8\pandas\plotting
  copying pandas\plotting\__init__.py -> build\lib.win-amd64-3.8\pandas\plotting
  creating build\lib.win-amd64-3.8\pandas\tests
  copying pandas\tests\test_algos.py -> build\lib.win-amd64-3.8\pandas\tests
  copying pandas\tests\test_base.py -> build\lib.win-amd64-3.8\pandas\tests
  copying pandas\tests\test_common.py -> build\lib.win-amd64-3.8\pandas\tests
  copying pandas\tests\test_compat.py -> build\lib.win-amd64-3.8\pandas\tests
  copying pandas\tests\test_config.py -> build\lib.win-amd64-3.8\pandas\tests
  

In [6]:
pip install Cython


  creating build\lib.win-amd64-3.8\pandas\tests\api
  copying pandas\tests\api\test_api.py -> build\lib.win-amd64-3.8\pandas\tests\api
  copying pandas\tests\api\test_types.py -> build\lib.win-amd64-3.8\pandas\tests\api
  copying pandas\tests\api\__init__.py -> build\lib.win-amd64-3.8\pandas\tests\api
  creating build\lib.win-amd64-3.8\pandas\tests\arithmetic
  copying pandas\tests\arithmetic\conftest.py -> build\lib.win-amd64-3.8\pandas\tests\arithmetic
  copying pandas\tests\arithmetic\test_datetime64.py -> build\lib.win-amd64-3.8\pandas\tests\arithmetic
  copying pandas\tests\arithmetic\test_numeric.py -> build\lib.win-amd64-3.8\pandas\tests\arithmetic
  copying pandas\tests\arithmetic\test_object.py -> build\lib.win-amd64-3.8\pandas\tests\arithmetic
  copying pandas\tests\arithmetic\test_period.py -> build\lib.win-amd64-3.8\pandas\tests\arithmetic
  copying pandas\tests\arithmetic\test_timedelta64.py -> build\lib.win-amd64-3.8\pandas\tests\arithmetic
  copying pandas\tests\arithme

  copying pandas\tests\extension\base\base.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\casting.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\constructors.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\dtype.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\getitem.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\groupby.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\interface.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\io.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\methods.py -> build\lib.win-amd64-3.8\pandas\tests\extension\base
  copying pandas\tests\extension\base\missing.py -> build\lib.win-a

  copying pandas\tests\series\test_repr.py -> build\lib.win-amd64-3.8\pandas\tests\series
  copying pandas\tests\series\test_sorting.py -> build\lib.win-amd64-3.8\pandas\tests\series
  copying pandas\tests\series\test_subclass.py -> build\lib.win-amd64-3.8\pandas\tests\series
  copying pandas\tests\series\test_timeseries.py -> build\lib.win-amd64-3.8\pandas\tests\series
  copying pandas\tests\series\test_timezones.py -> build\lib.win-amd64-3.8\pandas\tests\series
  copying pandas\tests\series\test_validate.py -> build\lib.win-amd64-3.8\pandas\tests\series
  copying pandas\tests\series\__init__.py -> build\lib.win-amd64-3.8\pandas\tests\series
  creating build\lib.win-amd64-3.8\pandas\tests\sparse
  copying pandas\tests\sparse\common.py -> build\lib.win-amd64-3.8\pandas\tests\sparse
  copying pandas\tests\sparse\test_combine_concat.py -> build\lib.win-amd64-3.8\pandas\tests\sparse
  copying pandas\tests\sparse\test_format.py -> build\lib.win-amd64-3.8\pandas\tests\sparse
  copying panda

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'G:\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [7]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'google.colab'

We will start by connecting to Kaggle using Kaggle API which can be downloaded from your Kaggle account's settings and uploading it here(upload box).

In [10]:
# Next, install the Kaggle API client.
!pip install -q kaggle

Next we will install Kaggle API using pip installation.

In [11]:
# The Kaggle API client expects this file to be in ~/.kaggle, so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


Setting up Kaggle using Kaggle API.

In [ ]:
# Creating directory and changing the current working directory
!mkdir dog_dataset
%cd dog_dataset

To store the data we will create a new directory and make it as current working directory.

In [ ]:
# Searching for dataset
!kaggle datasets list -s dogbreedidfromcomp

Searching Kaggle for the required dataset using search option(-s) with title 'dogbreedidfromcomp'. We can also use different search options like searching competitions, notebooks, kernels, datasets, etc.

In [ ]:
# Downloading dataset and coming out of directory
!kaggle datasets download catherinehorng/dogbreedidfromcomp
%cd ..

After searching the data next step would be downloading the data into collab notebook using references found in search option.

In [ ]:
# Unzipping downloaded file and removing unusable file
!unzip dog_dataset/dogbreedidfromcomp.zip -d dog_dataset
!rm dog_dataset/dogbreedidfromcomp.zip
!rm dog_dataset/sample_submission.csv

We will unzip the data which is downloaded and remove the irrelevant files. 

In [ ]:
# Important library imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam

Importing required libraries.

In [ ]:
# Read the labels.csv file and checking shape and records
labels_all = pd.read_csv("dog_dataset/labels.csv")
print(labels_all.shape)
labels_all.head()

Loading the labels data into dataframe and viewing it. Here we analysed that labels contains 10222 rows and 2 columns.  

In [ ]:
# Visualize the number of each breeds
breeds_all = labels_all["breed"]
breed_counts = breeds_all.value_counts()
breed_counts.head()

Here we are finding out the count per class i.e. total data in each class using value_counts() function.

In [ ]:
# Selecting first 3 breeds (Limitation due to computation power)
CLASS_NAMES = ['scottish_deerhound','maltese_dog','bernese_mountain_dog']
labels = labels_all[(labels_all['breed'].isin(CLASS_NAMES))]
labels = labels.reset_index()
labels.head()

We will work on only 3 breeds due to limited computational power. You can consider more classes as per your system computational power.

In [ ]:
# Creating numpy matrix with zeros
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float32')
# One hot encoding
Y_data = label_binarize(labels['breed'], classes = CLASS_NAMES)

# Reading and converting image to numpy array and normalizing dataset
for i in tqdm(range(len(labels))):
    img = image.load_img('dog_dataset/train/%s.jpg' % labels['id'][i], target_size=(224, 224))
    img = image.img_to_array(img)
    x = np.expand_dims(img.copy(), axis=0)
    X_data[i] = x / 255.0
    
# Printing train image and one hot encode shape & size
print('\nTrain Images shape: ',X_data.shape,' size: {:,}'.format(X_data.size))
print('One-hot encoded output shape: ',Y_data.shape,' size: {:,}'.format(Y_data.size))

As we are working with the classification dataset first we need to one hot encode the target value i.e. the classes. After that we will read images and convert them into numpy array and finally normalizing the array.

In [ ]:
# Building the Model
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5), activation ='relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 16, kernel_size = (7,7), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 8, kernel_size = (5,5), activation ='relu', kernel_regularizer = 'l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(64, activation = "relu", kernel_regularizer = 'l2'))
model.add(Dense(len(CLASS_NAMES), activation = "softmax"))

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

model.summary()

Next we will create a network architecture for the model. We have used different types of layers according to their features namely Conv_2d (It is used to create a convolutional kernel that is convolved with the input layer to produce the output tensor), max_pooling2d (It is a downsampling technique which takes out the maximum value over the window defined by poolsize), flatten (It flattens the input and creates a 1D output), Dense (Dense layer produce the output as the dot product of input and kernel).


After defining the network architecture we found out the total parameters as 162,619.

In [ ]:
# Splitting the data set into training and testing data sets
X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(X_data, Y_data, test_size = 0.1)
# Splitting the training data set into training and validation data sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train_and_val, Y_train_and_val, test_size = 0.2)

After defining the network architecture we will start with splitting the test and train data then dividing train data in train and validation data. 

In [ ]:
# Training the model
epochs = 100
batch_size = 128

history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, 
                    validation_data = (X_val, Y_val))

Now we will train our model on 100 epochs and a batch size of 128. You can try using more number of epochs to increase accuracy. During each epochs we can see how the model is performing by viewing the training and validation accuracy.

In [ ]:
# Plot the training history
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], color='r')
plt.plot(history.history['val_accuracy'], color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'val'])

plt.show()

Here we analyse how the model is learning with each epoch in terms of accuracy.

In [ ]:
Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test)
print('Accuracy over the test set: \n ', round((score[1]*100), 2), '%')

We will use predict function to make predictions using this model also we are finding out the accuracy on the test set.

In [ ]:
# Plotting image to compare
plt.imshow(X_test[1,:,:,:])
plt.show()

# Finding max value from predition list and comaparing original value vs predicted
print("Originally : ",labels['breed'][np.argmax(Y_test[1])])
print("Predicted : ",labels['breed'][np.argmax(Y_pred[1])])

Here you can see image with its original and predicted label.

## Conclusion:

We started with downloading the dataset creating the model and finding out the predictions using the model. We can optimize different hyper parameters in order to tune this model for a higher accuracy. This model can be used to predict different breeds of dogs which can be further used by different NGO's working on saving animals and for educational purposes also.